<a href="https://colab.research.google.com/github/sankeashok/NLP/blob/master/SL_ECT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.dropbox.com/s/5bhhs3hpyiawhel/ect_02_data.csv

--2020-11-25 06:48:44--  https://www.dropbox.com/s/5bhhs3hpyiawhel/ect_02_data.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5bhhs3hpyiawhel/ect_02_data.csv [following]
--2020-11-25 06:48:45--  https://www.dropbox.com/s/raw/5bhhs3hpyiawhel/ect_02_data.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc83bc21fd9dfde0657cc61451d3.dl.dropboxusercontent.com/cd/0/inline/BD0dchFVHg-8bNw47hMqYs8_O1axsccd259wjlLOFv5iQY_v_5mwieOuCcfjpbOpa2vp613tNBia8lTO_8wNOQ4egk1Xl1Lp3I3EPFfh4CnsTuIIoEuW809AZIiOvUlWQvc/file# [following]
--2020-11-25 06:48:45--  https://uc83bc21fd9dfde0657cc61451d3.dl.dropboxusercontent.com/cd/0/inline/BD0dchFVHg-8bNw47hMqYs8_O1axsccd259wjlLOFv5iQY_v_5mwieOuCcfjpbOpa2vp613tNBia8lTO_8wNOQ4egk1Xl1Lp3I3EPF

In [ ]:
import os
os.chdir = '/content/drive/MyDrive/AIML-NITW/6.SL/ECT2'

In [ ]:
!dir

drive  ect_02_data.csv	sample_data


In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout, Input, Flatten
from keras.callbacks import ModelCheckpoint

# Question 1.1
    . Remove everything except number and character from text
    . Tokenize data
    . all text lowercase

In [ ]:
data = pd.read_csv('ect_02_data.csv').iloc[:, 0:3]

In [ ]:
data.head()

,response_id,class,response_text
0,response_1,not_flagged,I try and avoid this sort of conflict
1,response_2,flagged,Had a friend open up to me about his mental ad...
2,response_3,flagged,I saved a girl from suicide once. She was goin...
3,response_4,not_flagged,i cant think of one really...i think i may hav...
4,response_5,not_flagged,Only really one friend who doesn't fit into th...


In [ ]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
    text = word_tokenize(text)
    text = [word.lower() for word in text]
    return ' '.join(text)

In [ ]:
import nltk
nltk.download('punkt')

data['clean_text'] = data.response_text.apply(clean_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data['clean_text']

0                 i try and avoid this sort of conflict
1     had a friend open up to me about his mental ad...
2     i saved a girl from suicide once she was going...
3     i cant think of one really i think i may have ...
4     only really one friend who doesn t fit into th...
                            ...                        
75    now that i ve been through it although i m not...
76    when my best friends mom past away from od ing...
77    as a camp counselor i provide stability in kid...
78    my now girlfriend used to have serious addicti...
79    the one person i ever talked to it was because...
Name: clean_text, Length: 80, dtype: object

# Question 1.2
    . Tokenize using Keras tokenizer
    . Convert text to sequence (texts_to_sequences)
    . Pad the sequences to streamline (pad_sequences)
    . Convert all to one_hot_encoding (OneHotEncoder)

In [ ]:
def lstm_data(text, labels):
    token = Tokenizer(10000)
    token.fit_on_texts(text)
    tokenized_data = token.texts_to_sequences(text)
    tokenized_data = pad_sequences(tokenized_data, maxlen= 20, padding= "post")
    encode_target = OneHotEncoder(sparse = False).fit(np.array(labels).reshape(-1,1))
    labels_data = encode_target.transform(np.array(labels).reshape(-1,1))
    return token, tokenized_data, labels_data

In [ ]:
tokenizer, text, labels = lstm_data(data.response_text.values, data['class'].values)

In [ ]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.
    
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

In [ ]:
text = transform_matrix(text, tokenizer)

In [ ]:
text

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0.

# Question 1.3 - Model
    . Embedding layer
    . LSTM cells (128)
    . Dense (2)
    . Dropout (0.5)
    . Dense (2)

In [ ]:
text.shape

(80, 20, 677)

In [ ]:
def lstm_model():
    ip_layer = Input(shape=(text.shape[1],))
    embedding_layer = Embedding(input_dim=tokenizer.word_index.keys().__len__()+1,output_dim=128,input_length=20)(ip_layer)
    lstm_layer = LSTM(units= 128)(embedding_layer)
    dense_layer_1= Dense(2, activation='relu')(lstm_layer)
    drop_layer = Dropout(0.5)(dense_layer_1)
    final_layer = Dense(2, activation='sigmoid')(drop_layer)
    return Model(inputs= [ip_layer], outputs= [final_layer])

In [ ]:
lstm_model().summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 128)           86784     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
_________________________________________________________________
dropout (Dropout)            (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6         
Total params: 218,632
Trainable params: 218,632
Non-trainable params: 0
________________________________________________

# Question 1.4 - Model Training
    . Optimizer: Adam
    . Epochs: 500
    . Batch size: 30
    . Save model as model.h5

In [ ]:
model= lstm_model()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(text, labels, epochs=500, batch_size=30, verbose=2)

In [ ]:
def lstm_data1(text, labels):
    token = Tokenizer(filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
    token.fit_on_texts(text)
    tokenized_data = token.texts_to_sequences(text)
    tokenized_data = pad_sequences(tokenized_data, maxlen= 20, padding= "post")
    encode_target = OneHotEncoder(sparse = False).fit(np.array(labels).reshape(-1,1))
    labels_data = encode_target.transform(np.array(labels).reshape(-1,1))
    return token, tokenized_data, labels_data

In [ ]:
tokenizer1, text1, labels1 = lstm_data1(data.response_text.values, data['class'].values)

In [ ]:
text1.shape

In [ ]:
def lstm_model1():
    ip_layer = Input(shape=(text.shape[1],))
    embedding_layer = Embedding(input_dim=tokenizer.word_index.keys().__len__()+1,output_dim=128,input_length=20)(ip_layer)
    lstm_layer = Bidirectional(LSTM(128))(embedding_layer)
    dense_layer_1= Dense(64, activation='relu')(lstm_layer)
    drop_layer = Dropout(0.5)(dense_layer_1)
    final_layer = Dense(2, activation='sigmoid')(drop_layer)
    return Model(inputs= [ip_layer], outputs= [final_layer])

In [ ]:
lstm_model1().summary()

In [ ]:
model1= lstm_model1()

In [ ]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model1.fit(text, labels, epochs=800, batch_size=40, verbose=2)